<a href="https://colab.research.google.com/github/RipeanuMihai/Digital-Reconstruction-of-Monuments-with-Generative-AI/blob/main/pheonix_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers controlnet-aux -q
!pip install gradio
!pip install hf_xet


SyntaxError: invalid syntax (ipython-input-2877666387.py, line 5)

In [ ]:
import torch
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
import gc
from diffusers import StableDiffusionXLControlNetPipeline, ControlNetModel
from controlnet_aux import MidasDetector


if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()


device = "cuda" if torch.cuda.is_available() else "cpu"


depth_detector = MidasDetector.from_pretrained("lllyasviel/Annotators")
controlnet = ControlNetModel.from_pretrained(
    "diffusers/controlnet-depth-sdxl-1.0",
    torch_dtype=torch.float16
)
pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to(device)

styles = {
    "Roman": "ancient Roman architecture",
    "Grec": "classical Greek temple with marble columns",
    "Gotic": "gothic medieval architecture with tall arches",
    "Egiptean": "ancient Egyptian monument with sandstone",
    "Mayas": "Mayan architecture with stone carvings and stepped terraces",
    "Renascentist": "Renaissance-style palace with domes and arches",
    "Islamic": "Islamic architecture with arches and geometric tiles",
    "Japonez": "Japanese wooden shrine style with minimalism",
    "Chinezesc": "traditional Chinese architecture with curved rooftops"
}

materials = {
    "Piatra": "built from stone",
    "Marmura": "white marble finish",
    "Caramida": "brick exterior",
    "Teracota": "terracotta tiles and ornamentation",
    "Lemn": "wooden structures with carved beams",
    "Granit": "polished granite surfaces"
}

monument_types = {
    "Templu": "ancient temple",
    "Amfiteatru": "colossal amphitheater",
    "Palat": "royal palace",
    "Fortareata": "stone fortress",
    "Piramida": "pyramid",
    "Statue": "statue",
    "Ziduri de aparare": "defensive stone walls",
    "Monument funerar": "ancient funerary monument"
}

negative_prompt = "blurry, distorted, cartoon, asymmetrical, text, low quality"

def add_watermark(image):
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    text = "Generated by AI"
    width, height = image.size
    position = (width - 10 - len(text) * 6, height - 20)
    draw.text(position, text, fill=(255, 255, 255), font=font)
    return image

def reconstruct(image, style, material, monument, guidance, show_depth):
    image = image.convert("RGB").resize((1024, 1024))
    depth_map = depth_detector(image).convert("RGB")

    prompt = f"Reconstructed monument, {styles[style]}, {materials[material]}, {monument_types[monument]}, photorealistic, symmetrical, ultra detailed"

    result = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=image,
        control_image=depth_map,
        controlnet_conditioning_scale=0.6,
        num_inference_steps=30,
        guidance_scale=guidance
    ).images[0]
    result = add_watermark(result)
    return result, depth_map if show_depth else None


gr.Interface(
    fn=reconstruct,
    inputs=[
        gr.Image(type="pil", label="Imagine deteriorata"),
        gr.Radio(choices=list(styles.keys()), label="Stil arhitectural"),
        gr.Radio(choices=list(materials.keys()), label="Material"),
        gr.Radio(choices=list(monument_types.keys()), label="Tip de monument"),
        gr.Slider(5, 15, value=12, step=1, label="Guidance scale"),
        gr.Checkbox(label="Afiseaza depth map-ul")
    ],
    outputs=[
        gr.Image(type="pil", label="Reconstructie finala"),
        gr.Image(type="pil", label="Harta de adancime (depth map)")
    ],
    title="Reconstructie Monumente",
    description="Reconstruieste automat monumente vechi"
).launch()

/usr/local/lib/python3.11/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.11/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segmen

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e957e989663ff8a7f7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
